# Eigenvektoren an Rahmentragwerk mit Nachgiebigkeitsmatrix

[Aufgabenstellung](calculations/%C3%9Cbung%201_2_MMS.pdf)

![Statisches System des 2-Massenschwingers](pictures/mms1.jpg){#fig-system}

In [1]:
import sympy as sp 
from sympycalcs import render, convert
import sympy.physics.units as unit

import matplotlib.pyplot as plt

## Parameter der Aufgabenstellung


In [2]:
params = {'EI': 20*10**12 *unit.N*unit.mm**2,
          'm_1':1000*unit.N*unit.second**2/unit.m,
          'm_2':1000*unit.N*unit.second**2/unit.m,
          'l':4*unit.m,
          }

render.dict_render(params)

Eq(EI, 20000000000000*millimeter**2*newton)

Eq(m_1, 1000*newton*second**2/meter)

Eq(m_2, 1000*newton*second**2/meter)

Eq(l, 4*meter)

In [3]:
omega, t,  l, m_1, m_2, EI = sp.symbols('omega, t,  l, m_1, m_2, EI')

omega_n = sp.symbols('omega_n')

## Nachgiebigkeitsmatrix $\mathbf{D}$ {#sec-nachgiebigkeit}

Die Steifigkeitsmatrix lässt sich durch Invertierung der Nachgiebigkeitsmatrix beschreiben. Die Nachgiebigkeitsmatrix $\mathbf{D}$ beschreibt die Deformation an einem Massenpunkt. Die Einträge der $\mathbf{D}$ - Matrix beschreiben die Deformationen für unterschiedliche Laststellungen.

$$\mathbf{K} = \mathbf{D^{-1}}$$

![Balken mit 2 Einzelmassen](pictures/bsp_nachgib.jpg){#fig-2mms}

Für einen 2-Massenschwinger, wie in @fig-2mms , hat die Nachgiebigkeitsmatrix folgende Form:

$$\mathbf{D} = \frac{1}{EI}*\begin{bmatrix}
\delta_{11} & \delta_{12}\\
\delta_{21} & \delta_{22} 
\end{bmatrix}$$

wobei gilt:

$\delta_{ab}$ : $a$ ist die Lastsituation, $b$ ist die Masse.


### Anwendung 

![Schnittgrössen für beide Lastfälle zur Bestimmung der Deformation](pictures/rahmen_def.jpg){#fig-schnittgrössen}

$$\delta_{ab} = \frac{1}{EI}\int_{0}^{l} M_a\bar{M_b} \,dx$$

Es werden 2 Laststellungen betrachtet, jeweils an einem Massenpunkt. Dabei ist Beachtung der Einheit der Einwirkung zu schenken. Diese wird einheitslos angesetzt.

In [4]:
delta_11 = 1/(EI) * (sp.Rational(1,3) * -l*-l*1*l)* 2
delta_12 = 1/(EI) * ((sp.Rational(1,3) * -l*-l*1*l) + (sp.Rational(1,2) * -l*-l*1*l))
delta_21 = delta_12
delta_22 = 1/(EI) * ((sp.Rational(1,3) * -l*-l*1*l)* 2 + -l*-l*1*l)

render.eq_display('delta_11', delta_11,
                  'delta_12', delta_12,
                  'delta_21', delta_21,
                  'delta_22', delta_22)


Eq(delta_11, 2*l**3/(3*EI))

Eq(delta_12, 5*l**3/(6*EI))

Eq(delta_21, 5*l**3/(6*EI))

Eq(delta_22, 5*l**3/(3*EI))

In [5]:
D = sp.Matrix([[delta_11, delta_12],[delta_21, delta_22]])
K = D.inv()
render.eq_display(sp.MatrixSymbol('D', 2,2), D,
                  sp.MatrixSymbol('K', 2,2), K)

Eq(D, Matrix([
[2*l**3/(3*EI), 5*l**3/(6*EI)],
[5*l**3/(6*EI), 5*l**3/(3*EI)]]))

Eq(K, Matrix([
[ 4*EI/l**3,    -2*EI/l**3],
[-2*EI/l**3, 8*EI/(5*l**3)]]))

## Eigenvektoren

Die Bewegungsgleichung für einen ungedämpften, frei schwingenden Mehrmassenschwinger lässt sich folgender massen aufstellen:

$$\mathbf{M u''(t) + K u(t)} = 0$$

Die Modale Analyse entkoppelt die Gleichungen aus der Matrixschreibweise um diese unabhängig von einander zu lösen.

### Massenmatrix $\mathbf{M}$

In [6]:
M = sp.Matrix([[m_1,0],[0,m_2]])

render.eq_display(sp.MatrixSymbol('M',2,2), M)

Eq(M, Matrix([
[m_1,   0],
[  0, m_2]]))

### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand folgender Gleichung bestimmen:

$$\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}$$

In [7]:
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

Eq(omega_1, 12.1/second)

Eq(omega_2, 40.0/second)

### Eigenvektoren $\mathbf{\phi}$

Dazu ist die entsprechende Normierung aus der Aufgabenstellung zu berücksichtigen.

In [8]:
phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')
params['phi_11'] = 1

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_21 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_21).values())[0]

params['phi_21'] = phi_21.subs(params).simplify() 




render.eq_display((K-omega_1**2 *M)*phi_1,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))


Eq(Matrix([
[    -2*EI*phi_21/l**3 + phi_11*(4*EI/l**3 - m_1*(4*EI/(5*l**3*m_2) + 2*EI/(l**3*m_1) - 2*EI*sqrt(4*m_1**2 + 5*m_1*m_2 + 25*m_2**2)/(5*l**3*m_1*m_2)))],
[-2*EI*phi_11/l**3 + phi_21*(8*EI/(5*l**3) - m_2*(4*EI/(5*l**3*m_2) + 2*EI/(l**3*m_1) - 2*EI*sqrt(4*m_1**2 + 5*m_1*m_2 + 25*m_2**2)/(5*l**3*m_1*m_2)))]]), Matrix([
[0],
[0]]))

Eq(phi_1, Matrix([
[ 1.0],
[1.77]]))

In [9]:
params['phi_12'] = 1

phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_22 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_22).values())[0]

params['phi_22'] = phi_22.subs(params).simplify() 




render.eq_display((K-omega_2**2 *M)*phi_2,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))


Eq(Matrix([
[    -2*EI*phi_22/l**3 + phi_12*(4*EI/l**3 - m_1*(4*EI/(5*l**3*m_2) + 2*EI/(l**3*m_1) + 2*EI*sqrt(4*m_1**2 + 5*m_1*m_2 + 25*m_2**2)/(5*l**3*m_1*m_2)))],
[-2*EI*phi_12/l**3 + phi_22*(8*EI/(5*l**3) - m_2*(4*EI/(5*l**3*m_2) + 2*EI/(l**3*m_1) + 2*EI*sqrt(4*m_1**2 + 5*m_1*m_2 + 25*m_2**2)/(5*l**3*m_1*m_2)))]]), Matrix([
[0],
[0]]))

Eq(phi_2, Matrix([
[   1.0],
[-0.566]]))

### Orthogonalitätsbedingung

Zur effektiven Entkoppelung der Gleichungen muss die Orthogonalitätsbedingung eingehalten sein. Dies gilt es für die Massenmatrix zu kontrollieren:

$$\mathbf{\phi_1^T M \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_1 = 0}$$

Sowohl auch für die Steifigkeitsmatrix:

$$\mathbf{\phi_1^T K \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_1 = 0}$$



In [10]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_2.T*M*phi_1).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(3))

Eq(phi_1.T*M*phi_1, Matrix([[4.12e+3*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_2, Matrix([[1.32e+3*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_1, Matrix([[0]]))

Eq(phi_1.T*M*phi_2, Matrix([[0]]))

In [11]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(3))

Eq(phi_1.T*K*phi_1, Matrix([[6.02e+5*newton/meter]]))

Eq(phi_2.T*K*phi_2, Matrix([[2.12e+6*newton/meter]]))

Eq(phi_2.T*K*phi_1, Matrix([[0]]))

Eq(phi_1.T*K*phi_2, Matrix([[0]]))

### Eigenformen

![Die beiden Eigenformen skizziert](pictures/eigenformen.jpg){#fig-eigenformen}